# Proyecto PAPIME PE110923 DESARROLLO DE UN LABORATORIO DE ROBOTICA REMOTO PARA REALIZAR PRACTICAS DE PROGRAMACION DE ALGORITMOS DE PLANEACION Y DE NAVEGACION EN BANCOS DE PRUEBA FISICOS

# Robot Móvil Diferencial
Autores: M.I. Erik Peña Medina, Ing. Felipe Rivas Campos y  Dr. Víctor Javier González Villela.

## Resumen

En este documento se presenta la configuración básica de un robot móvil diferencial (2,0) que cuenta con un sensor lidar y una cámara de profundidad. El contenido de este paquete es:

```text
 src/
    ├── diff_description/     
    |       ├── launch
    |       |       └── display_diff.launch.xml
    |       ├── rviz/
    |       |       ├── display_diff_config.rviz
    |       │       └── urdf.rviz
    |       ├── urdf/
    |       │       ├── camara.xacro
    |       │       ├── lidar.xacro
    |       │       ├── common_properties.xacro
    |       │       ├── material.xacro
    |       │       ├── mobile_base_gazebo.xacro
    |       │       ├── mobile_base.xacro
    |       │       └── my_robot.urdf.xacro
    |       ├── CMakeLists.txt
    |       └── package.xml
    └── diff_bringup/
            ├── rviz/
            │       └── diff_robot_gz_config.rviz
            ├── launch
            │       └── diff_robot_simulation.launch.xml
            ├── world/
            │       └── test_w_1.world
            ├── CMakeLists.txt
            └── package.xml
   
```

## Descripción del Robot Móvil (2,0)

Para simular un Robot Móvil Diferencial (2,0), es necesario que los componentes del modelo tengan una disposición específica de los elementos de colisión que describen sus llantas y su rueda loca. La descripción de los elementos físicos que componen al robot se basa en cuerpos con formas primitivas, lo cual facilita su comprensión y la aplicación del código en la descripción de otros robots diferenciales que se deseen simular. Además, es posible modificar sus mallas de visualización y ajustar el tamaño de las mallas de colisión según las necesidades del nuevo modelo.

La descripción del robot está compuesta por diferentes archivos Xacros, para simplificar la descripción de los elementos del robot. La descripción de algunos elementos que componen al robot son redundantes y para evitar tener que programalarlas en cada uno de estos se utilizarón funciones xacro:macro. A continuación, se presenta una descripción ordenada de los archivos que describen al robot.

### Archivo mobile_base.xacro

El archivo mobile_base.xacro describe al robot haciendo uso de funciones xacro:macro para definir la inercial del cuerpo del robot y de su rueda loca, y una función que define a cada una de las llantas.

Código del archivo:

```xml

<?xml version="1.0"?>
<!-- 
Definición de un robot móvil diferencial simple usando XACRO (XML Macros), 
permitiendo reutilizar código y hacer configuraciones modulares.
-->
<robot xmlns:xacro="http://www.ros.org/wiki/xacro">

    <!-- Definición de propiedades reutilizables (parametrización) -->
    <xacro:property name="base_length" value="0.6" />   <!-- Longitud del cuerpo base -->
    <xacro:property name="base_width" value="0.4" />    <!-- Ancho del cuerpo base -->
    <xacro:property name="base_height" value="0.2" />   <!-- Altura del cuerpo base -->
    <xacro:property name="wheel_radius" value="0.1" />  <!-- Radio de las ruedas -->
    <xacro:property name="wheel_length" value="0.05" /> <!-- Longitud de las ruedas -->

    <!-- Punto de referencia base para la simulación -->
    <link name="base_footprint" />

    <!-- Definición del cuerpo principal del robot -->
    <link name="base_link">
        <visual>
            <geometry>
                <!-- Geometría del cuerpo: Caja rectangular -->
                <box size="${base_length} ${base_width} ${base_height}" />
            </geometry>
            <!-- Posición de la caja con respecto al origen -->
            <origin xyz="0 0 ${base_height / 2.0}" rpy="0 0 0" />
            <material name="blue" /> <!-- Material visual -->
        </visual>
        <collision>
            <geometry>
                <!-- Geometría de colisión: Caja idéntica a la visual -->
                <box size="${base_length} ${base_width} ${base_height}" />
            </geometry>
            <origin xyz="0 0 ${base_height / 2.0}" rpy="0 0 0" />
        </collision>
        <!-- Inercia calculada para el cuerpo base -->
        <xacro:box_inertia m="5.0" l="${2*base_length}" w="${2*base_width}" h="${2*base_height}"
                           xyz="0 0 ${base_height / 2.0}" rpy="0 0 0" />
    </link>

    <!-- Macro reutilizable para definir las ruedas -->
    <xacro:macro name="wheel_link" params="prefix">
        <link name="${prefix}_wheel_link">
            <visual>
                <geometry>
                    <!-- Geometría de la rueda: Cilindro -->
                    <cylinder radius="${wheel_radius}" length="${wheel_length}" />
                </geometry>
                <!-- Orientación de la rueda -->
                <origin xyz="0 0 0" rpy="${pi / 2.0} 0 0" />
                <material name="grey" /> <!-- Material visual -->
            </visual>
            <collision>
                <geometry>
                    <!-- Geometría de colisión: Cilindro -->
                    <cylinder radius="${wheel_radius}" length="${wheel_length}" />
                </geometry>
                <origin xyz="0 0 0" rpy="${pi / 2.0} 0 0" />
            </collision>
            <!-- Inercia calculada para la rueda -->
            <xacro:cylinder_inertia m="1.0" r="${2*wheel_radius}" h="${2*wheel_length}" 
                                    xyz="0 0 0" rpy="${pi / 2.0} 0 0" />
        </link>
    </xacro:macro>

    <!-- Creación de las ruedas derecha e izquierda usando la macro -->
    <xacro:wheel_link prefix="right" />
    <xacro:wheel_link prefix="left" />

    <!-- Definición de la rueda loca (caster wheel) -->
    <link name="caster_wheel_link">
        <visual>
            <geometry>
                <!-- Geometría de la rueda loca: Esfera -->
                <sphere radius="${wheel_radius / 2.0}" />
            </geometry>
            <origin xyz="0 0 0" rpy="0 0 0" />
            <material name="grey" /> <!-- Material visual -->
        </visual>
        <collision>
            <geometry>
                <!-- Geometría de colisión: Esfera -->
                <sphere radius="${wheel_radius / 2.0}" />
            </geometry>
            <origin xyz="0 0 0" rpy="0 0 0" />
        </collision>
        <!-- Inercia calculada para la rueda loca -->
        <xacro:sphere_inertia m="0.5" r="${2*wheel_radius / 2.0}"
                              xyz="0 0 0" rpy="0 0 0" />
    </link>

    <!-- Joints (articulaciones) que conectan los diferentes componentes -->
    
    <!-- Junta fija que conecta el punto base con el cuerpo principal -->
    <joint name="base_joint" type="fixed">
        <parent link="base_footprint" />
        <child link="base_link" />
        <origin xyz="0 0 ${wheel_radius}" rpy="0 0 0"/>
    </joint>

    <!-- Junta continua para la rueda derecha (permite movimiento rotacional) -->
    <joint name="base_right_wheel_joint" type="continuous">
        <parent link="base_link" />
        <child link="right_wheel_link" />
        <!-- Posición de la rueda derecha respecto al cuerpo principal -->
        <origin xyz="${-base_length / 4.0} ${-(base_width + wheel_length) / 2.0} 0" rpy="0 0 0" />
        <axis xyz="0 1 0" /> <!-- Eje de rotación -->
    </joint>

    <!-- Junta continua para la rueda izquierda (permite movimiento rotacional) -->
    <joint name="base_left_wheel_joint" type="continuous">
        <parent link="base_link" />
        <child link="left_wheel_link" />
        <!-- Posición de la rueda izquierda respecto al cuerpo principal -->
        <origin xyz="${-base_length / 4.0} ${(base_width + wheel_length) / 2.0} 0" rpy="0 0 0" />
        <axis xyz="0 1 0" /> <!-- Eje de rotación -->
    </joint>

    <!-- Junta fija que conecta la rueda loca (caster wheel) -->
    <joint name="base_caster_wheel_joint" type="fixed">
        <parent link="base_link" />
        <child link="caster_wheel_link" />
        <!-- Posición de la rueda loca respecto al cuerpo principal -->
        <origin xyz="${base_length / 3.0} 0 ${-wheel_radius / 2.0}" rpy="0 0 0" />
    </joint>

</robot>

```
El código anterior se puede resumir lo siguiente:

1. Propiedades XACRO: Permiten parametrizar dimensiones clave como tamaño del cuerpo y ruedas.
2. Cuerpo principal (base_link): Una caja rectangular que define el cuerpo del robot con visualización, colisión e inercia.
3. Macro de rueda (wheel_link): Reutilizable para definir múltiples ruedas con geometría cilíndrica.
4. Rueda loca (caster_wheel_link): Se define como una esfera con una junta fija.
5. Articulaciones (joints): Conectan las partes del robot:
    - Junta fija para el cuerpo.
    - Juntas continuas para las ruedas motrices.
    - Junta fija para la rueda loca.

Este código es una representación modular y limpia de un robot móvil diferencial con dos ruedas motrices y una rueda loca.


### Archivos material.xacro y common_properties.xacro

El código de los archivos material.xacro y common_properties.xacro contiene los elementos que complementan las descripción del robot del archivo mobile_base.xacro. 

Código del archivo material.xacro:

```xml

<?xml version="1.0"?>
<!--
Este archivo define un robot llamado 'diff_robot' utilizando XACRO.
Incluye la definición de materiales y macros para calcular propiedades inerciales 
de elementos geométricos básicos: cajas, cilindros y esferas.
-->

<robot name="diff_robot" xmlns:xacro="http://www.ros.org/wiki/xacro">
    
    <!-- Definición de materiales para visualización -->
    <material name="blue">
        <color rgba="0 0 0.5 1" /> <!-- Color azul con transparencia nula -->
    </material>

    <material name="grey">
        <color rgba="0.5 0.5 0.5 1" /> <!-- Color gris neutro -->
    </material>

    <material name="orange">
        <color rgba="1.0 0.5 0 1" /> <!-- Color naranja -->
    </material>

    <material name="yellow">
        <color rgba="1.0 1.0 0 1" /> <!-- Color amarillo -->
    </material>

    <!-- Macro para calcular la inercia de una caja -->
    <xacro:macro name="box_inertia" params="m l w h xyz rpy">
        <!-- 
        Parámetros:
        m   - masa de la caja
        l   - longitud de la caja
        w   - ancho de la caja
        h   - altura de la caja
        xyz - posición del origen de la inercia (x, y, z)
        rpy - orientación del origen (roll, pitch, yaw)
        -->
        <inertial>
            <!-- Posición y orientación del origen de inercia -->
            <origin xyz="${xyz}" rpy="${rpy}" />
            <mass value="${m}" /> <!-- Masa de la caja -->
            <!-- Tensor de inercia, calculado con la fórmula clásica -->
            <inertia ixx="${(m/12) * (h*h + l*l)}" ixy="0" ixz="0"
                     iyy="${(m/12) * (w*w + l*l)}" iyz="0"
                     izz="${(m/12) * (w*w + h*h)}" />
        </inertial>
    </xacro:macro>

    <!-- Macro para calcular la inercia de un cilindro -->
    <xacro:macro name="cylinder_inertia" params="m r h xyz rpy">
        <!-- 
        Parámetros:
        m   - masa del cilindro
        r   - radio del cilindro
        h   - altura (longitud) del cilindro
        xyz - posición del origen de la inercia (x, y, z)
        rpy - orientación del origen (roll, pitch, yaw)
        -->
        <inertial>
            <!-- Posición y orientación del origen de inercia -->
            <origin xyz="${xyz}" rpy="${rpy}" />
            <mass value="${m}" /> <!-- Masa del cilindro -->
            <!-- Tensor de inercia, calculado para un cilindro -->
            <inertia ixx="${(m/12) * (3*r*r + h*h)}" ixy="0" ixz="0"
                     iyy="${(m/12) * (3*r*r + h*h)}" iyz="0"
                     izz="${(m/2) * (r*r)}" />
        </inertial>
    </xacro:macro>

    <!-- Macro para calcular la inercia de una esfera -->
    <xacro:macro name="sphere_inertia" params="m r xyz rpy">
        <!-- 
        Parámetros:
        m   - masa de la esfera
        r   - radio de la esfera
        xyz - posición del origen de la inercia (x, y, z)
        rpy - orientación del origen (roll, pitch, yaw)
        -->
        <inertial>
            <!-- Posición y orientación del origen de inercia -->
            <origin xyz="${xyz}" rpy="${rpy}" />
            <mass value="${m}" /> <!-- Masa de la esfera -->
            <!-- Tensor de inercia, calculado con la fórmula de una esfera homogénea -->
            <inertia ixx="${(2/5) * m * r * r}" ixy="0" ixz="0"
                     iyy="${(2/5) * m * r * r}" iyz="0"
                     izz="${(2/5) * m * r * r}" />
        </inertial>
    </xacro:macro>

</robot>

```

Resumen del Código:

1. Materiales:
    - Se definen cuatro materiales (blue, grey, orange, yellow) con colores RGBA. Estos se pueden aplicar a diferentes partes del robot en su definición.

2. Macros de Inercia:
    - box_inertia: Calcula la inercia para una caja. Utiliza las fórmulas clásicas de inercia para cajas en función de la masa, altura, ancho y longitud.
    - cylinder_inertia: Calcula la inercia de un cilindro sólido. La inercia depende del radio, longitud y masa.
    - sphere_inertia: Calcula la inercia de una esfera sólida homogénea. La fórmula utilizada es (2/5) * m * r² para los términos diagonales.

3. Uso de Parámetros:
    - Los parámetros xyz y rpy permiten definir la posición y orientación del origen de la inercia.
    - Los macros son reutilizables, lo que permite que cualquier parte del robot con forma de caja, cilindro o esfera tenga su inercia correctamente definida.

Este código es una excelente base modular y eficiente para definir las propiedades inerciales de componentes de un robot en URDF utilizando XACRO.


Código del archivo common_properties.xacro:

```xml

<?xml version="1.0"?>
<!--
Este archivo define un conjunto de materiales y macros para calcular 
la inercia de formas geométricas básicas usando XACRO.
-->

<robot xmlns:xacro="http://www.ros.org/wiki/xacro">
    
    <!-- Definición de materiales para visualización del robot -->
    <material name="blue">
        <!-- Material azul oscuro con opacidad completa -->
        <color rgba="0 0 0.5 1" />
    </material>

    <material name="grey">
        <!-- Material gris neutro con opacidad completa -->
        <color rgba="0.5 0.5 0.5 1" />
    </material>

    <material name="orange">
        <!-- Material naranja brillante con opacidad completa -->
        <color rgba="1.0 0.5 0 1" />
    </material>

    <material name="yellow">
        <!-- Material amarillo brillante con opacidad completa -->
        <color rgba="1.0 1.0 0 1" />
    </material>

    <!--
    Macro: box_inertia
    Descripción:
        Calcula la inercia de una caja con masa uniforme.
    Parámetros:
        m    - masa de la caja
        l    - longitud de la caja
        w    - ancho de la caja
        h    - altura de la caja
        xyz  - posición del centro de masa (x, y, z)
        rpy  - orientación del centro de masa (roll, pitch, yaw)
    -->
    <xacro:macro name="box_inertia" params="m l w h xyz rpy">
        <inertial>
            <!-- Posición y orientación del centro de masa -->
            <origin xyz="${xyz}" rpy="${rpy}" />
            <!-- Masa de la caja -->
            <mass value="${m}" />
            <!-- Tensor de inercia (diagonal), calculado con fórmula clásica -->
            <inertia ixx="${(m/12) * (h*h + l*l)}" ixy="0" ixz="0"
                     iyy="${(m/12) * (w*w + l*l)}" iyz="0"
                     izz="${(m/12) * (w*w + h*h)}" />
        </inertial>
    </xacro:macro>

    <!--
    Macro: cylinder_inertia
    Descripción:
        Calcula la inercia de un cilindro con masa uniforme.
    Parámetros:
        m    - masa del cilindro
        r    - radio del cilindro
        h    - altura del cilindro
        xyz  - posición del centro de masa (x, y, z)
        rpy  - orientación del centro de masa (roll, pitch, yaw)
    -->
    <xacro:macro name="cylinder_inertia" params="m r h xyz rpy">
        <inertial>
            <!-- Posición y orientación del centro de masa -->
            <origin xyz="${xyz}" rpy="${rpy}" />
            <!-- Masa del cilindro -->
            <mass value="${m}" />
            <!-- Tensor de inercia (diagonal), usando fórmula específica para cilindros -->
            <inertia ixx="${(m/12) * (3*r*r + h*h)}" ixy="0" ixz="0"
                     iyy="${(m/12) * (3*r*r + h*h)}" iyz="0"
                     izz="${(m/2) * (r*r)}" />
        </inertial>
    </xacro:macro>

    <!--
    Macro: sphere_inertia
    Descripción:
        Calcula la inercia de una esfera con masa uniforme.
    Parámetros:
        m    - masa de la esfera
        r    - radio de la esfera
        xyz  - posición del centro de masa (x, y, z)
        rpy  - orientación del centro de masa (roll, pitch, yaw)
    -->
    <xacro:macro name="sphere_inertia" params="m r xyz rpy">
        <inertial>
            <!-- Posición y orientación del centro de masa -->
            <origin xyz="${xyz}" rpy="${rpy}" />
            <!-- Masa de la esfera -->
            <mass value="${m}" />
            <!-- Tensor de inercia (esfera sólida homogénea), igual en todas las direcciones -->
            <inertia ixx="${(2/5) * m * r * r}" ixy="0" ixz="0"
                     iyy="${(2/5) * m * r * r}" iyz="0"
                     izz="${(2/5) * m * r * r}" />
        </inertial>
    </xacro:macro>

</robot>

```

Resumen de Comentarios:

1. Materiales:
    - Cuatro materiales (blue, grey, orange, yellow) se definen usando valores RGBA.
    - Estos materiales son útiles para aplicar colores a las partes visuales del robot.

2. Macros para inercia:
    - box_inertia: Calcula la inercia de una caja uniforme usando su masa, longitud, ancho y altura.
    - cylinder_inertia: Calcula la inercia de un cilindro uniforme, tomando en cuenta el radio y la altura.
    - sphere_inertia: Calcula la inercia de una esfera uniforme, cuyo tensor es isotrópico (igual en todas direcciones).

3. Reutilización de código:
    - El uso de macros hace que este archivo sea modular y eficiente.
    - Se pueden reutilizar estas macros en otras partes del modelo URDF/XACRO para definir rápidamente las propiedades físicas de formas geométricas simples.

Este archivo es una plantilla versátil para definir propiedades inerciales y materiales de robots en URDF utilizando XACRO, lo cual es esencial para la simulación física y visualización.

## Elementos de Simulación para Gazebo

Para simular al robot dentro de un ambiente en Gazebo se le agregan las definiciones de los elementos físicos del robot y la definición de una cámara de profundidad y de la sensor lidar. 

### Archivo mobile_base_gazebo.xacro

El archivo mobile_base_gazebo.xacro contiene las definiciones de los elementos físicos del robot, además, del plugin que define al controlador del robot móvil diferencial.

Código del archivo mobile_base_gazebo.xacro:

```xml

<?xml version="1.0"?>
<!-- 
Este archivo define configuraciones específicas de simulación en Gazebo 
para un robot diferencial usando XACRO. 
-->

<robot xmlns:xacro="http://www.ros.org/wiki/xacro">

    <!-- Configuración de la apariencia visual de la base del robot en Gazebo -->
    <gazebo reference="base_link">
        <!-- Asigna el color azul predefinido en Gazebo -->
        <material>Gazebo/Blue</material>
    </gazebo>

    <!-- Configuración para la rueda derecha -->
    <gazebo reference="right_wheel_link">
        <!-- Asigna el color gris predefinido en Gazebo -->
        <material>Gazebo/Grey</material>
        <!-- Coeficientes de fricción en dos direcciones -->
        <mu1 value="0.1" /> <!-- Fricción en la dirección principal -->
        <mu2 value="0.1" /> <!-- Fricción en la dirección secundaria -->
    </gazebo>

    <!-- Configuración para la rueda izquierda -->
    <gazebo reference="left_wheel_link">
        <!-- Asigna el color gris predefinido en Gazebo -->
        <material>Gazebo/Grey</material>
        <!-- Coeficientes de fricción en dos direcciones -->
        <mu1 value="0.1" />
        <mu2 value="0.1" />
    </gazebo>

    <!-- Configuración para la rueda loca (caster) -->
    <gazebo reference="caster_wheel_link">
        <!-- Asigna el color gris predefinido en Gazebo -->
        <material>Gazebo/Grey</material>
        <!-- Coeficientes de fricción en dos direcciones -->
        <mu1 value="0.1" />
        <mu2 value="0.1" />
    </gazebo>

    <!-- Configuración del plugin para controlar el movimiento diferencial -->
    <gazebo>
        <plugin name="diff_drive_controller" filename="libgazebo_ros_diff_drive.so">
            <!-- Tasa de actualización del controlador en Hertz -->
            <update_rate>50</update_rate>

            <!-- Especifica las articulaciones asociadas a las ruedas -->
            <left_joint>base_left_wheel_joint</left_joint>
            <right_joint>base_right_wheel_joint</right_joint>

            <!-- Configuración de parámetros cinemáticos -->
            <wheel_separation>0.45</wheel_separation> <!-- Separación entre ruedas -->
            <wheel_diameter>0.2</wheel_diameter>       <!-- Diámetro de las ruedas -->

            <!-- Configuración de salida de la odometría -->
            <publish_odom>true</publish_odom> <!-- Publicar datos de odometría -->
            <publish_odom_tf>true</publish_odom_tf> <!-- Publicar transformación odom_tf -->
            <publish_wheel_tf>true</publish_wheel_tf> <!-- Publicar transformación de las ruedas -->
            <odometry_topic>odom</odometry_topic> <!-- Nombre del tópico para la odometría -->
            <odometry_frame>odom</odometry_frame> <!-- Marco de referencia de odometría -->
            <robot_base_frame>base_footprint</robot_base_frame> <!-- Marco base del robot -->
        </plugin>
    </gazebo>

</robot>

```
Resumen del código:

1. Materiales de las partes:
    - Se configuran los colores de la base del robot (blue) y las ruedas (grey) utilizando materiales predefinidos de Gazebo.
    - Los materiales facilitan la visualización en la simulación.

2. Coeficientes de fricción (mu1 y mu2):
    - Se definen bajos valores de fricción para las ruedas, lo cual simula un movimiento más suave y adecuado para un robot diferencial.

3. Plugin diff_drive_controller:
    - Propósito: Controlar el movimiento diferencial de las ruedas.
    - Parámetros clave:
        - update_rate: Frecuencia de actualización del controlador.
        - left_joint y right_joint: Articulaciones de las ruedas izquierda y derecha.
        - wheel_separation y wheel_diameter: Configuración geométrica del robot para calcular correctamente la odometría y movimiento.
4. Publicaciones:
    - Odometría: Se publica en el tópico odom.
    - Transformaciones: Publica tanto la transformación de odometría (odom_tf) como las transformaciones de las ruedas.

### Sensor Lidar

Para el sensor lidar se agrega el cuerpo cilindrico que representa al sensor, el código que define a este sensor es:

```xml
<?xml version="1.0"?>
<!-- Archivo XACRO que define un sensor LIDAR montado sobre un robot -->
<robot xmlns:xacro="http://www.ros.org/wiki/xacro">

    <!-- Definición del enlace físico del LIDAR -->
    <link name="lidar_link">
        <!-- Sección visual: cómo se verá el LIDAR en la simulación -->
        <visual>
            <geometry>
                <cylinder radius="0.1" length="0.05" /> <!-- Cilindro para representar el LIDAR -->
            </geometry>
            <origin xyz="0 0 0" rpy="0 0 0" /> <!-- Posición y orientación del visual -->
            <material name="grey" /> <!-- Color gris -->
        </visual>

        <!-- Sección de colisión: define el volumen físico para las interacciones -->
        <collision>
            <geometry>
                <cylinder radius="0.1" length="0.05" />
            </geometry>
            <origin xyz="0 0 0" rpy="0 0 0" />
        </collision>

        <!-- Sección inercial: define la masa y las inercias del LIDAR -->
        <xacro:cylinder_inertia m="0.1" r="0.1" h="0.05" 
                                xyz="0 0 0" rpy="0 0 0" />
    </link>

    <!-- Enlace lógico del sensor LIDAR (para asociarlo con el plugin) -->
    <link name="lidar_sensor" />

    <!-- Junta fija que conecta el LIDAR al cuerpo principal del robot -->
    <joint name="lidar_joint" type="fixed">
        <parent link="base_link" /> <!-- La base del robot -->
        <child link="lidar_link" /> <!-- Enlace del LIDAR -->
        <!-- Posiciona el LIDAR sobre la base -->
        <origin xyz="0 0 ${base_height+0.05/2}" rpy="0 0 0" />
    </joint>

    <!-- Junta fija entre el enlace visual LIDAR y el sensor lógico -->
    <joint name="lidar_sensor_joint" type="fixed">
        <parent link="lidar_link" />
        <child link="lidar_sensor" />
        <origin xyz="0 0 0" rpy="0 0 0" />
    </joint>

    <!-- Configuración del sensor LIDAR en Gazebo -->
    <gazebo reference="lidar_sensor">
        <sensor type="ray" name="lidar">
            <pose>0 0 0 0 0 0</pose> <!-- Posición relativa dentro del enlace -->
            <visualize>true</visualize> <!-- Visualizar los rayos del sensor -->
            <always_on>true</always_on> <!-- El sensor siempre está activo -->
            <update_rate>15</update_rate> <!-- Tasa de actualización en Hz -->

            <!-- Configuración del rayo LIDAR -->
            <ray>
                <!-- Configuración de escaneo -->
                <scan>
                    <horizontal>
                        <samples>360</samples> <!-- Número de muestras -->
                        <resolution>1</resolution> <!-- Resolución del sensor -->
                        <min_angle>-3.14</min_angle> <!-- Ángulo mínimo (radianes) -->
                        <max_angle>3.14</max_angle> <!-- Ángulo máximo (radianes) -->
                    </horizontal>
                </scan>

                <!-- Configuración del rango de detección -->
                <range>
                    <min>0.15</min> <!-- Rango mínimo en metros -->
                    <max>1.5</max> <!-- Rango máximo en metros -->
                    <resolution>0.017453</resolution> <!-- Resolución angular (~1 grado) -->
                </range>

                <!-- Configuración del ruido del sensor -->
                <noise>
                    <type>gaussian</type> <!-- Tipo de ruido: Gaussiano -->
                    <mean>0.0</mean> <!-- Media del ruido -->
                    <stddev>0.01</stddev> <!-- Desviación estándar del ruido -->
                </noise>
            </ray>

            <!-- Plugin para integrar el sensor LIDAR en ROS -->
            <plugin name="laser_plugin" filename="libgazebo_ros_ray_sensor.so">
                <ros>
                    <!-- Remapeo del tópico donde se publicarán los datos del LIDAR -->
                    <remapping>~/out:=scan</remapping>
                </ros>
                <!-- Tipo de mensaje que se publica -->
                <output_type>sensor_msgs/LaserScan</output_type>
                <!-- Frame de referencia para el sensor -->
                <frame_name>lidar_sensor</frame_name>
            </plugin>
        </sensor>
    </gazebo>

</robot>

```

Explicación General del Código

1. Enlace lidar_link:
    - Representa el cuerpo físico del LIDAR en forma de cilindro.
        Se definen sus propiedades visual (cómo se ve), colisión (interacción física) e inercial (masa e inercias).

2. Enlace lidar_sensor:
    - Es un enlace lógico que asocia el sensor al modelo y permite usarlo con Gazebo.

3. Juntas Fijas (fixed):
    - La primera junta (lidar_joint) conecta el LIDAR a la base del robot, posicionándolo justo encima.
    - La segunda junta (lidar_sensor_joint) conecta el sensor lógico con el cuerpo físico.

4. Configuración del Sensor LIDAR:
    - Tipo: ray, para simular un sensor láser que emite rayos.
        - Parámetros del rayo:
            - Escaneo horizontal con 360 muestras (1 grado por muestra).
            - Ángulo de escaneo completo de -π a π (360 grados).
            - Rango de detección entre 0.15 m y 1.5 m.
        - Ruido: Se aplica ruido gaussiano con desviación estándar de 0.01.

5. Plugin libgazebo_ros_ray_sensor.so:
    - Publica los datos del LIDAR como mensajes sensor_msgs/LaserScan.
    - Se remapea el tópico a scan.
    - El frame de referencia del sensor es lidar_sensor.

Uso Final

Este código agrega un sensor LIDAR funcional en la simulación de Gazebo que:

- Realiza un escaneo láser de 360 grados.
- Publica los datos en formato ROS para ser utilizados en algoritmos de navegación o percepción.
- Simula ruido y actualiza a una frecuencia de 15 Hz.


### Cámara de profundidad

La cámara de profundidad cuenta con un rayo laser el cual genera una nube de puntos los cuales se emplean para hacer una rescontrucción del entorno. En la definición cuenta con un camera_link_optical, donde el eje óptico de la cámara está asociado con el eje Z, por lo que el sistema de referencia tiene una orientación específica. 

El código que define la cámara de profundidad es:

```xml

<?xml version="1.0"?>
<!-- Definición de un sensor de cámara montado en el robot -->
<robot xmlns:xacro="http://www.ros.org/wiki/xacro">

    <!-- Propiedades geométricas de la cámara -->
    <xacro:property name="camera_length" value="0.01" />
    <xacro:property name="camera_width" value="0.1" />
    <xacro:property name="camera_height" value="0.05" />

    <!-- Enlace físico para el cuerpo de la cámara -->
    <link name="camera_link">
        <!-- Representación visual de la cámara -->
        <visual>
            <geometry>
                <box size="${camera_length} ${camera_width} ${camera_height}" />
            </geometry>
            <material name="grey" /> <!-- Color gris -->
        </visual>
        
        <!-- Volumen de colisión de la cámara -->
        <collision>
            <geometry>
                <box size="${camera_length} ${camera_width} ${camera_height}" />
            </geometry>
        </collision>

        <!-- Propiedades inerciales para la cámara -->
        <xacro:box_inertia m="0.1" l="${camera_length}" w="${camera_width}" h="${camera_height}"
                           xyz="0 0 0" rpy="0 0 0" />
    </link>

    <!-- Junta fija que monta la cámara sobre el cuerpo base -->
    <joint name="base_camera_joint" type="fixed">
        <parent link="base_link" />
        <child link="camera_link" />
        <!-- Posición de la cámara respecto al centro del robot -->
        <origin xyz="${(base_length + camera_length) / 2.0} 0 ${base_height / 2.0}" rpy="0 0 0" />
    </joint>

    <!-- Enlace lógico para la cámara óptica (orientación correcta en Gazebo) -->
    <link name="camera_link_optical">
    </link>

    <!-- Junta fija que alinea el frame de la cámara óptica -->
    <joint name="camera_optical_joint" type="fixed">
        <!-- Es necesario ajustar la orientación para la compatibilidad con Gazebo -->
        <origin xyz="0 0 0" rpy="${-pi/2} 0 ${-pi/2}"/>
        <parent link="camera_link"/>
        <child link="camera_link_optical"/>
    </joint>

    <!-- Configuración visual de la cámara en Gazebo -->
    <gazebo reference="camera_link">
        <material>Gazebo/Red</material>
    </gazebo>

    <!-- Configuración del sensor de profundidad -->
    <gazebo reference="camera_link">
        <sensor name="camera_sensor" type="depth">
            <pose>0 0 0 0 0 0</pose> <!-- Posición y orientación del sensor -->
            <visualize>true</visualize> <!-- Mostrar visualmente los datos del sensor -->
            <update_rate>10.0</update_rate> <!-- Frecuencia de actualización en Hz -->
            <camera>
                <format>R8G8B8</format> <!-- Formato de imagen: RGB -->
                <width>640</width> <!-- Ancho de la imagen en píxeles -->
                <height>480</height> <!-- Alto de la imagen en píxeles -->
            </camera>
            <!-- Configuración de la distancia de recorte (cercana y lejana) -->
            <clip>
                <near>0.05</near> <!-- Distancia mínima en metros -->
                <far>8.0</far> <!-- Distancia máxima en metros -->
            </clip>

            <!-- Plugin para publicar los datos de la cámara en ROS -->
            <plugin name="camera_controller" filename="libgazebo_ros_camera.so">
                <frame_name>camera_link_optical</frame_name> <!-- Frame de referencia -->
            </plugin>
        </sensor>
    </gazebo>

</robot>

```

Resumen del código:1

1. Propiedades de la Cámara:
    - Se definen las dimensiones de la cámara usando propiedades (camera_length, camera_width, camera_height) para mayor modularidad.

2. Enlace camera_link:
    - Representa el cuerpo físico de la cámara como un box.
    - Incluye propiedades visual (cómo se ve), colisión (interacción física) e inercial (masa e inercias).

3. Junta base_camera_joint:
    - Conecta la cámara con el cuerpo base (base_link) del robot.
    - Se posiciona la cámara en el frente y por encima del cuerpo del robot.

4. Frame camera_link_optical:
    - Este frame es necesario para que Gazebo oriente correctamente la cámara en el espacio.
    - Se aplica una rotación de -π/2 alrededor de los ejes X y Z para ajustarse al estándar de las imágenes en Gazebo.

5. Sensor depth en Gazebo:
    - Tipo de sensor: depth para simular una cámara de profundidad.
    - Parámetros de la imagen:
        - Formato RGB.
        - Resolución de 640x480 píxeles.
    - Distancia de recorte:
        - Rango entre 0.05 m (cercano) y 8.0 m (lejano).

6. Plugin libgazebo_ros_camera.so:
    - Publica los datos de la cámara en formato ROS.
    - El frame de referencia es camera_link_optical.

Uso Final

- La cámara simula una cámara de profundidad que publica imágenes en formato RGB.
- Los datos se pueden usar en aplicaciones de visión artificial o navegación en ROS2.
- El sensor está configurado para actualizarse a 10 Hz y simula un rango efectivo de 0.05 m a 8.0 m.

Con esta configuración, puedes visualizar la cámara en Gazebo, recibir imágenes en tiempo real en ROS, y utilizar tópicos como camera/image_raw para procesar los datos.



## Descripción total del robot

La descripción total del robot se conforma en el el archivo my_robot.urdf.xacro:

```xml
<?xml version="1.0"?>
<!-- Definición del robot diferencial con estructura modular -->
<robot name="diff_robot" xmlns:xacro="http://www.ros.org/wiki/xacro">

    <!-- Inclusión de propiedades comunes -->
    <xacro:include filename="common_properties.xacro" />
    <!-- common_properties.xacro: Contiene propiedades generales, como colores, materiales, y constantes geométricas. -->

    <!-- Inclusión de la base móvil -->
    <xacro:include filename="mobile_base.xacro" />
    <!-- mobile_base.xacro: Define la estructura física de la base del robot (base_link, ruedas, juntas y otras geometrías). -->

    <!-- Configuración de la base para Gazebo -->
    <xacro:include filename="mobile_base_gazebo.xacro" />
    <!-- mobile_base_gazebo.xacro: Contiene los plugins y configuraciones necesarias para la simulación en Gazebo.
         Ejemplos: plugins de control diferencial, fricción de las ruedas, o parámetros de físicas. -->

    <!-- Inclusión del modelo de la cámara -->
    <xacro:include filename="camera.xacro" />
    <!-- camera.xacro: Define el modelo físico y el sensor de la cámara, que puede incluir una cámara RGB o de profundidad.
         Contiene las dimensiones, propiedades inerciales y plugins de Gazebo para publicar las imágenes. -->

    <!-- Inclusión del modelo del LiDAR -->
    <xacro:include filename="lidar.xacro" />
    <!-- lidar.xacro: Define el modelo físico y el sensor LiDAR.
         Contiene las especificaciones del sensor tipo "ray", rango de escaneo y ruido, además del plugin de ROS
         para publicar datos del sensor en el tópico correspondiente. -->

</robot>
```

Explicación General

1. Uso de xacro:include:
    - La directiva <xacro:include> permite modularizar el código y mantener organizado el modelo del robot.
    - Cada archivo .xacro encapsula una parte específica del robot:
        - common_properties.xacro: Materiales, colores y constantes geométricas.
        - mobile_base.xacro: La estructura principal de la base móvil, como enlaces y juntas.
        - mobile_base_gazebo.xacro: Configuraciones específicas para Gazebo, como plugins y propiedades físicas.
        - camera.xacro: La definición del sensor de cámara.
            lidar.xacro: La definición del sensor LiDAR.

2. Estructura Modular:
    - Dividir el código en módulos facilita el mantenimiento y la reutilización.
    - Si es necesario cambiar las propiedades de un sensor o la base móvil, solo se modifica el archivo correspondiente sin afectar el modelo principal.

3. Arquitectura Recomendada:
    - Un archivo principal (diff_robot.xacro) reúne todas las partes del robot.
    - Archivos modulares definen componentes específicos del robot.

Ventajas de Esta Organización

- Reutilización: Puedes reutilizar módulos (como camera.xacro o lidar.xacro) en otros robots o simulaciones.
- Legibilidad: Facilita la comprensión del modelo principal porque cada archivo se enfoca en un subsistema específico.
- Mantenimiento: Los cambios en una parte del robot no afectan al resto del modelo.
- Escalabilidad: Permite añadir nuevas funcionalidades (como sensores adicionales) sin desorganizar el archivo principal.


### Configuración CMakeLists.txt

La configuración del archivo CMakeLists.txt para el paquete diff_description es:
```txt

cmake_minimum_required(VERSION 3.8)
project(diff_description)

if(CMAKE_COMPILER_IS_GNUCXX OR CMAKE_CXX_COMPILER_ID MATCHES "Clang")
  add_compile_options(-Wall -Wextra -Wpedantic)
endif()

# find dependencies
find_package(ament_cmake REQUIRED)

install(
  DIRECTORY urdf launch rviz
  DESTINATION share/${PROJECT_NAME}/
)

ament_package()

```

Para este caso no es necesario agregar elementos al archivo package.xml.


### Visualización del robot

Para vizualizar el robot se utiliza el archivo display_diff.launch.xml en la carpeta launch y el archivo display_diff_config.rviz de la carpeta de rviz.

Código de archivo display_diff.launch.xml:


```xml
<launch>
    <!-- Define la ruta del archivo URDF (Universal Robot Description Format) del robot -->
    <let name="urdf_path" 
         value="$(find-pkg-share diff_description)/urdf/my_robot.urdf.xacro" />
    <!-- Define la ruta del archivo de configuración de RViz para la visualización del robot -->
    <let name="rviz_config_path"
         value="$(find-pkg-share diff_description)/rviz/display_diff_config.rviz" />

    <!-- Nodo para publicar el estado del robot (estado de las juntas) -->
    <node pkg="robot_state_publisher" exec="robot_state_publisher">
        <param name="robot_description"
               value="$(command 'xacro $(var urdf_path)')" />
    </node>

    <!-- Nodo para controlar el estado de las juntas en un GUI -->
    <node pkg="joint_state_publisher_gui" exec="joint_state_publisher_gui" />

    <!-- Nodo para lanzar RViz2 con la configuración definida -->
    <node pkg="rviz2" exec="rviz2" output="screen" 
          args="-d $(var rviz_config_path)" />
</launch>

```

Explicación Detallada

1. Definición de Variables (<let>):
    - urdf_path: Utiliza el comando find-pkg-share para encontrar el paquete diff_description y busca el archivo my_robot.urdf.xacro dentro de la carpeta urdf. Este archivo contiene la descripción del robot en formato URDF (o XACRO, que es un formato dinámico para URDF).
    - rviz_config_path: Similarmente, busca la configuración de RViz en el paquete diff_description, específicamente en el archivo display_diff_config.rviz. Este archivo contiene la configuración y disposición de las vistas en RViz.

2. Nodo robot_state_publisher:
    - Este nodo publica el estado del robot, específicamente el estado de sus juntas, basado en el archivo de descripción del robot (robot_description).
    - La línea value="$(command 'xacro $(var urdf_path)')" ejecuta el archivo .xacro para obtener la descripción completa del robot en formato URDF, y lo pasa como parámetro al nodo robot_state_publisher. Este nodo genera y publica los mensajes de estado de las juntas en el tópico correspondiente.

3. Nodo joint_state_publisher_gui:

    - Este nodo proporciona una interfaz gráfica para interactuar y visualizar los estados de las juntas del robot. A través de este GUI, puedes mover las juntas del robot y ver los cambios reflejados en RViz.

4. Nodo rviz2:
    - Lanza RViz2 (la herramienta de visualización en ROS2).
    - La opción args="-d $(var rviz_config_path)" carga el archivo de configuración de RViz previamente definido, lo que permite personalizar la visualización del robot (por ejemplo, qué elementos mostrar, las cámaras, sensores, etc.).

Flujo de Ejecución

- Paso 1: Cuando el archivo de lanzamiento se ejecuta, las variables urdf_path y rviz_config_path se definen con las rutas correctas de los archivos URDF (robot description) y de configuración de RViz.
- Paso 2: Se ejecuta el nodo robot_state_publisher, que utiliza la descripción del robot en formato URDF (generado por XACRO) para publicar el estado de las juntas en los tópicos correspondientes.
- Paso 3: El nodo joint_state_publisher_gui proporciona una interfaz gráfica para manipular las juntas del robot.
- Paso 4: RViz2 se lanza con la configuración especificada, permitiendo la visualización del robot y sus sensores.


El archivo display_diff_config.rviz carga la lectura de los tópicos para vizualizar al robot.

![mobil_diff_1.png](imagenes/mobil_diff_1.png)


## Simulación en Gazebo

La simulación que se presenta en Gazebo coloca al robot (2,0) en un ambiente de pruebas en el cual este puede transladarse de un lugar a otro. Para la simulación se requiere el ambiente test_w_1.world dentro de la carpeta world y el archivo diff_robot_simulation.launch.xml, el archivo diff_robot_gz_config.rviz leé la información de los tópicos generados por los sensores y el plugin de control del robot. 

Código del archivo diff_robot_simulation.launch.xml:

```xml
<launch>
    <!-- Define la ruta del archivo URDF (o XACRO) que describe el robot -->
    <let name="urdf_path" 
         value="$(find-pkg-share diff_description)/urdf/my_robot.urdf.xacro" />

    <!-- Define la ruta del archivo de configuración de RViz para el robot -->
    <let name="rviz_config_path"
         value="$(find-pkg-share diff_bringup)/rviz/diff_robot_gz_config.rviz" />

    <!-- Nodo para publicar el estado de las juntas del robot -->
    <node pkg="robot_state_publisher" exec="robot_state_publisher">
        <param name="robot_description"
               value="$(command 'xacro $(var urdf_path)')" />
    </node>

    <!-- Incluir el archivo de lanzamiento de Gazebo para cargar el mundo y la simulación -->
    <include file="$(find-pkg-share gazebo_ros)/launch/gazebo.launch.py">
     <arg name="world" value="$(find-pkg-share diff_bringup)/world/test_w_1.world" />
    </include>

    <!-- Nodo para spawn (generar) el robot en Gazebo usando la descripción del robot -->
    <node pkg="gazebo_ros" exec="spawn_entity.py"
          args="-topic robot_description -entity diff_robot" />

    <!-- Nodo para lanzar RViz y mostrar la configuración del robot -->
    <node pkg="rviz2" exec="rviz2" output="screen" 
          args="-d $(var rviz_config_path)" />
</launch>

```

Explicación Detallada

1. Definición de Variables (<let>):
    - urdf_path: Se utiliza el comando find-pkg-share para encontrar el paquete diff_description y buscar el archivo my_robot.urdf.xacro. Este archivo contiene la descripción del robot en formato XACRO (que es un formato dinámico para URDF).
    - rviz_config_path: Encuentra el archivo de configuración de RViz diff_robot_gz_config.rviz en el paquete diff_bringup. Este archivo tiene configuraciones específicas de visualización del robot.

2. Nodo robot_state_publisher:
    - Este nodo se encarga de publicar el estado del robot. La descripción del robot se genera dinámicamente al ejecutar el archivo XACRO (con el comando xacro), lo que le permite publicar los mensajes de estado de las juntas para que otros nodos puedan utilizar esa información, como rviz2 o gazebo.

3. Incluir el archivo de lanzamiento de Gazebo:
    - El archivo gazebo.launch.py se incluye en el archivo de lanzamiento, lo que inicia Gazebo con el mundo predeterminado configurado en el archivo de argumentos world. Este archivo especifica un mundo en Gazebo (test_w_1.world) que se encuentra en el paquete diff_bringup.

4. Nodo spawn_entity.py:
    - Este nodo usa el archivo URDF de descripción del robot (en este caso, desde el tópico robot_description) y genera la entidad del robot en el simulador Gazebo. El robot se genera con el nombre diff_robot.

5. Nodo rviz2:
    - Este nodo lanza RViz2 y carga la configuración visual definida en el archivo diff_robot_gz_config.rviz. Este archivo puede incluir configuraciones sobre cómo se visualiza el robot, los sensores, las cámaras, etc., en RViz.

Flujo de Ejecución

- Paso 1: Se definen las variables urdf_path y rviz_config_path que especifican las rutas de los archivos necesarios para la descripción del robot y la configuración de RViz.
- Paso 2: Se lanza el nodo robot_state_publisher que publica la descripción del robot en el sistema de ROS2 a partir del archivo XACRO.
- Paso 3: Se incluye el archivo de lanzamiento de Gazebo, lo que lanza Gazebo con un mundo específico (test_w_1.world), que define el entorno en el que el robot va a simularse.
- Paso 4: El nodo spawn_entity.py toma la descripción del robot y genera el robot dentro del entorno de Gazebo.
- Paso 5: Finalmente, el nodo rviz2 abre RViz y carga la configuración visual del robot desde el archivo de configuración especificado.


## Simulación

El robot móvil simulado en Gazebo se presenta en la siguientes imagenes:
![mobil_diff_2.png](imagenes/mobil_diff_2.png)

![mobil_diff_3.png](imagenes/mobil_diff_3.png)


La lectura de los tópicos del robot para administrar la información que recibe el robot de su estado actual y de su entorno utilizando RVIZ se muestra en la siguiente imagen:

![mobil_diff_4.png](imagenes/mobil_diff_4.png)

Ya construida la simulación se deben generar los siguientes tópicos:

![mobil_diff_5.png](imagenes/mobil_diff_5.png)


## Conclusiones

Se presentan los archivos necesarios para realizar la simulación de un robot móvil diferencial (2,0), la cual cuenta con un control para el robot y sensores que permiten conocer el estado del robot y el estado de su entorno. Para la descripción del robot se emplearon archivos Xacro y se generarón funciones xacro:macro lo que dio versatilidad a la progrmación del robot. 

En otro documento se presentará el manejo del robot utilizando mensajes tipo twist.




